In [36]:
## Set up

# imports
import sys
sys.path.append('../../')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dont_add.config.api_key import weather_key as wKey
import requests
from scipy.stats import linregress
from pprint import pprint
from citipy import citipy
import random
import datetime
# from code.python.project_fns import saveTimeStamped

# paths
out_table_path_base = os.path.join('..','..','output','tables')
out_graph_path_base = os.path.join('..','..','output','graphs')
out_cities_list_path = os.path.join('..','..','data','cities')


In [41]:
def saveTimeStamped(df, base_path, name):
    name = '/' + name
    time_stamp = datetime.date.today()
    file_name = f'{name}_{time_stamp}'
    outpath = base_path + file_name
    df.to_csv(outpath, index=False)
    return

In [33]:
def makeCitiesList(country_coords):
    
    # Range of latitudes and longitudes for the set confines
    lat_range = (country_coords['lat'][0], country_coords['lat'][1])
    lng_range = (country_coords['lng'][0], country_coords['lng'][1])

    # List for holding lat_lngs and cities
    lat_lngs = []
    cities = []

    # Create a set of random lat and lng combinations
    lats = np.random.uniform(low=lat_range[0], high=lat_range[1], size=1500)
    lngs = np.random.uniform(low=lng_range[0], high=lng_range[1], size=1500)
    lat_lngs = zip(lats, lngs)

    # Identify nearest city for each lat, lng combination
    for lat_lng in lat_lngs:
        city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

        # If the city is unique, then add it to a our cities list
        if city not in cities:
            cities.append(city)

    # Print the city count to confirm sufficient count
    len(cities)
    return(cities)

In [43]:
def collectData(cities, filename):
    
    # Lists to store data into
    cur_temps = []
    min_temps = []
    max_temps = []
    feel_temps = []
    humidities = []
    cloudinessess = []
    windSpeeds = []
    cityLats = []
    cityLngs = []
    dates = []
    countries = []
    names = []
    pressures = []
    
    # loop to get the data for each city
    units = 'imperial'
    count = 1
    print('Beginning Data Retrieval')
    print('--------------------------------------')
    for city in cities:
        params = {'q' : city, 'appid': wKey, 'units' : units}
        url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={wKey}&units=imperial'
        response = requests.get(url).json()
        print(f'Attempting to save data for {city} : (number{count})')
        try:
            names.append(response['name'])
            max_temps.append(response['main']['temp_max'])
            cur_temps.append(response['main']['temp'])
            min_temps.append(response['main']['temp_min'])
            feel_temps.append(response['main']['feels_like'])
            humidities.append(response['main']['humidity'])
            cloudinessess.append(response['clouds']['all'])
            windSpeeds.append(response['wind']['speed'])
            cityLats.append(response['coord']['lat'])
            cityLngs.append(response['coord']['lon'])
            dates.append(response['dt'])
            countries.append(response['sys']['country'])
            pressures.append(response['main']['pressure'])
        except KeyError:
            print(f'failed to find all info for {city}, skipping remaining info')
        count += 1
    print('--------------------------------------')
    print('Data Retrieval Complete')
    print('--------------------------------------')
    
    # make Pandas DF for collected data
    base_df_dict = {
        'City' : names,
        'Cloudiness' : cloudinessess,
        'Country' : countries,
        'Date' : dates,
        'Humidity' : humidities,
        'Lat' : cityLats,
        'Lng' : cityLngs,
        'Current Temp' : cur_temps,
        'Max Temp' : max_temps,
        'Min Temp' : min_temps,
        'Feels Like Temp' : feel_temps,
        'Wind Speed' : windSpeeds,
        'Pressure' : pressures}

    base_df = pd.DataFrame(base_df_dict)    
    saveTimeStamped(df=base_df, base_path=out_table_path_base, name=filename)
    return


In [40]:
# collectData(cities=test_cities, filename='test')

In [45]:
BAHA = {'lat' : [20.94982, 26.86667],
       'lng' : [-79.3, -72.96667]}

BHAH_cities = makeCitiesList(BAHA)

In [46]:
print(BHAH_cities)

['cockburn town', 'clarence town', 'andros town', 'florida', 'rock sound', 'dunmore town', 'yaguajay', 'manati', 'primero de enero', 'baracoa', 'alice town', 'marsh harbour', 'venezuela', 'moron', 'nuevitas', 'minas', 'freeport', 'moa', 'gibara', 'esmeralda', 'caibarien', 'nassau', 'cockburn harbour', 'high rock', 'chambas', 'vertientes', 'sibanicu', 'banes', 'baragua', 'amancio', 'key largo', 'ciego de avila', 'port-de-paix', 'jatibonico', 'sancti spiritus', 'guaimaro', 'puerto padre', 'las tunas', 'sagua la grande', 'sagua de tanamo', 'jobabo', 'camaguey', 'colombia']


In [47]:
collectData(cities=BHAH_cities, filename='Bahamas')

Beginning Data Retrieval
--------------------------------------
Attempting to save data for cockburn town : (number1)
Attempting to save data for clarence town : (number2)
Attempting to save data for andros town : (number3)
Attempting to save data for florida : (number4)
Attempting to save data for rock sound : (number5)
Attempting to save data for dunmore town : (number6)
Attempting to save data for yaguajay : (number7)
Attempting to save data for manati : (number8)
Attempting to save data for primero de enero : (number9)
Attempting to save data for baracoa : (number10)
Attempting to save data for alice town : (number11)
Attempting to save data for marsh harbour : (number12)
Attempting to save data for venezuela : (number13)
Attempting to save data for moron : (number14)
Attempting to save data for nuevitas : (number15)
Attempting to save data for minas : (number16)
Attempting to save data for freeport : (number17)
Attempting to save data for moa : (number18)
Attempting to save data f